In [1]:
import os
import pandas as pd

train_path = "train/train/"
test_path = "test/test/"

train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

print(f"Jumlah file train: {len(train_files)}")
print(f"Jumlah file test: {len(test_files)}")

Jumlah file train: 4257
Jumlah file test: 2057


In [2]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np

model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features.flatten()

# Ekstrak fitur untuk semua gambar train
train_features = []
for file in train_files:
    features = extract_features(os.path.join(train_path, file))
    train_features.append(features)

train_features = np.array(train_features)

C:\Users\andri\AppData\Local\Temp\ipykernel_5732\646103949.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 997ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━

In [3]:
from sklearn.cluster import KMeans

n_clusters = 15  # sesuai jumlah kategori makanan
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(train_features)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Model untuk klasifikasi dari fitur yang sudah diekstraksi
feature_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(15, activation='softmax')
])

feature_model.compile(optimizer='adam', 
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])

c:\Users\andri\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
import joblib

# Simpan model
feature_model.save('food_classifier_model.h5')
joblib.dump(le, 'label_encoder.pkl')
joblib.dump(kmeans, 'kmeans_model.pkl')

# Simpan juga model feature extractor jika perlu
model.save('feature_extractor.h5')

NameError: name 'le' is not defined

In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.models import load_model
import joblib

# Muat model dan encoder
feature_model = load_model('food_classifier_model.h5')
le = joblib.load('label_encoder.pkl')

# Baca test.csv
test_df = pd.read_csv("test.csv")

# Pastikan test_files sesuai dengan urutan di test.csv
test_files = [f"{id}.jpg" for id in test_df['ID']]  # Sesuaikan dengan format nama file

test_path = "test/test/"
predictions = []

for file in test_files:
    img_path = os.path.join(test_path, file)
    
    # Ekstrak fitur
    features = extract_features(img_path)
    
    # Prediksi dengan model klasifikasi
    pred = feature_model.predict(np.expand_dims(features, axis=0), verbose=0)
    label_idx = np.argmax(pred)
    
    # Decode label
    label_name = le.inverse_transform([label_idx])[0]
    predictions.append(label_name)

# Assign ke test_df
test_df['label'] = predictions

# Simpan submission
test_df.to_csv("submission.csv", index=False)
print("Submission file created successfully!")
print(test_df.head())

Found 3406 validated image filenames belonging to 15 classes.
Found 851 validated image filenames belonging to 15 classes.
Epoch 1/15
107/107 ━━━━━━━━━━━━━━━━━━━━ 51s 471ms/step - accuracy: 0.0854 - loss: 3.3292 - val_accuracy: 0.1093 - val_loss: 2.6390
Epoch 2/15
107/107 ━━━━━━━━━━━━━━━━━━━━ 50s 467ms/step - accuracy: 0.1239 - loss: 2.6238 - val_accuracy: 0.1868 - val_loss: 2.5969
Epoch 3/15
107/107 ━━━━━━━━━━━━━━━━━━━━ 51s 475ms/step - accuracy: 0.2120 - loss: 2.4606 - val_accuracy: 0.2244 - val_loss: 2.4865
Epoch 4/15
107/107 ━━━━━━━━━━━━━━━━━━━━ 51s 474ms/step - accuracy: 0.3914 - loss: 1.9849 - val_accuracy: 0.2397 - val_loss: 2.4207
Epoch 5/15
107/107 ━━━━━━━━━━━━━━━━━━━━ 50s 468ms/step - accuracy: 0.5590 - loss: 1.4200 - val_accuracy: 0.2256 - val_loss: 2.5035
Epoch 6/15
107/107 ━━━━━━━━━━━━━━━━━━━━ 50s 471ms/step - accuracy: 0.6970 - loss: 0.9991 - val_accuracy: 0.2503 - val_loss: 2.5994
Epoch 7/15
107/107 ━━━━━━━━━━━━━━━━━━━━ 50s 466ms/step - accuracy: 0.7660 - loss: 0.7450 - 

In [ ]:
test_df = pd.read_csv("test.csv")
predictions = []

for file in test_files:
    img_path = os.path.join(test_path, file)
    features = extract_features(img_path)
    # Gunakan model klasifikasi untuk prediksi
    pred = model.predict(np.expand_dims(features, axis=0))
    label_idx = np.argmax(pred)
    predictions.append(label_names[label_idx])

test_df['label'] = predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>